In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from src.components.memory.memory import Memory
from src.components.collector.collector import AppDataCollector, LLMResponseCollector
from src.components.models.llms.llms import Langchain_OpenAI_LLM, HF_Llama38b_LLM
from src.components.models.embeddings.embeddings import HF_MultilingualE5_Embeddings, Openai_Embeddings
from src.components.memory.memory import Memory

# Iniciando modelos LLM
langchain_llm = Langchain_OpenAI_LLM()
langchain_llm.init_model()
hf_llm = HF_Llama38b_LLM()
hf_llm.init_model()

# Iniciando modelos Embeddings
mle5_embeddings = HF_MultilingualE5_Embeddings()
mle5_embeddings.init_model()

openai_embeddings = Openai_Embeddings()
openai_embeddings.init_model()

# Iniciando memoria
memory = Memory()

In [2]:
from src.app.pipeline_processes.query_post_process.manager import query_post_process
from src.app.pipeline_processes.query_pre_process.manager import (
    query_pre_process,
    simple_request_process,
)
from src.app.pipeline_processes.sql_generation_process.manager import (
    complex_request_sql_generation,
)
from src.app.pipeline_processes.sql_post_process.manager import (
    complex_request_pre_query_generation,
    complex_request_sql_summary_response,
    complex_request_sql_verification,
)
from src.app.pipeline_processes.sql_pre_process.manager import (
    complex_request_process_modification,
    complex_request_process_semantics,
)
from src.utils.sql_utils import run_sql
import traceback
import pandas as pd

def pre_process_pipeline(
    memory: Memory, collector: AppDataCollector, llm_collector: LLMResponseCollector
):
    try:
        query_pre_process(
            llm=hf_llm, memory=memory, collector=collector, llm_collector=llm_collector
        )
    except Exception as e:
        print(f"Error en: query_pre_process:\n {e}")
        traceback.print_exc()


def simple_request_pipeline(
    memory: Memory, collector: AppDataCollector, llm_collector: LLMResponseCollector
):
    try:
        simple_request_process(
            llm=langchain_llm,
            memory=memory,
            collector=collector,
            llm_collector=llm_collector,
        )
    except Exception as e:
        print(f"Error en: simple_request_process:\n {e}")
        traceback.print_exc()


def complex_request_pipeline(
    collector: AppDataCollector, llm_collector: LLMResponseCollector, memory: Memory
):
    try:
        print("complex_request_process_modification")
        complex_request_process_modification(
            llm=hf_llm,
            embeddings=mle5_embeddings,
            memory=memory,
            collector=collector,
            llm_collector=llm_collector,
        )
    except Exception as e:
        print(f"Error en: complex_request_process_modification:\n {e}")
        traceback.print_exc()

    try:
        print("complex_request_process_semantics")
        complex_request_process_semantics(
            llm=hf_llm,
            embeddings=openai_embeddings,
            collector=collector,
            llm_collector=llm_collector,
        )
    except Exception as e:
        print(f"Error en: complex_request_process_semantics:\n {e}")
        traceback.print_exc()

    try:
        print("complex_request_sql_generation")
        complex_request_sql_generation(
            llm=langchain_llm, collector=collector, llm_collector=llm_collector
        )
    except Exception as e:
        print(f"Error en: complex_request_sql_generation:\n {e}")
        traceback.print_exc()

    try:
        print("complex_request_sql_verification")
        complex_request_sql_verification(
            llm=hf_llm, collector=collector, llm_collector=llm_collector
        )
    except Exception as e:
        print(f"Error en: complex_request_sql_verification:\n {e}")
        traceback.print_exc()

    is_prequery = collector.assistant_sql_code_class.strip() == "incomplete"
    if is_prequery:
        try:
            print("complex_request_pre_query_generation")
            complex_request_pre_query_generation(
                llm=langchain_llm, collector=collector, llm_collector=llm_collector
            )
        except Exception as e:
            print(f"Error en: complex_request_pre_query_generation:\n {e}")
            traceback.print_exc()


def post_process_pipeline(
    user_message: str, collector: AppDataCollector, llm_collector: LLMResponseCollector
):
    df = None
    is_prequery = (
        collector.assistant_sql_code_class.strip() == "incomplete"
        if collector.assistant_sql_code_class
        else False
    )

    if is_prequery:
        try:
            print("Ejecutando codigo SQL de prequery ...")
            df = run_sql(collector.sql_pre_query)
        except Exception as e:
            print(f"Error al ejecutar collector.sql_pre_query \n{e}")
            traceback.print_exc()
    else:
        try:
            print("Ejecutando codigo SQL ...")
            df = run_sql(collector.sql_code)
        except Exception as e:
            print(f"Error al ejecutar collector.sql_code \n{e}")
            traceback.print_exc()

        try:
            complex_request_sql_summary_response(
                llm=langchain_llm,
                collector=collector,
                llm_collector=llm_collector,
                dataframe=df,
            )
        except Exception as e:
            print(f"Error al ejecutar complex_request_sql_summary_response\n{e}")
            traceback.print_exc()

    query_post_process(
        llm=langchain_llm,
        collector=collector,
        user_message=user_message,
        llm_collector=llm_collector,
    )


def generate_sql(
    memory: Memory,
    collector: AppDataCollector,
    llm_collector: LLMResponseCollector,
    user_message: str,
):


    pre_process_pipeline(
        memory=memory, collector=collector, llm_collector=llm_collector
    )
    is_simple = collector.request_type.lower().strip() == "simple"

    if is_simple:
        simple_request_pipeline(
            memory=memory, collector=collector, llm_collector=llm_collector
        )

    else:
        complex_request_pipeline(
            collector=collector, llm_collector=llm_collector, memory=memory
        )

    post_process_pipeline(
        user_message=user_message, collector=collector, llm_collector=llm_collector
    )



def chat(
    user_message: str,
    memory: Memory,
    collector: AppDataCollector,
    llm_collector: LLMResponseCollector,
):
    memory.add_user_message(user_message)
    generate_sql(
        memory=memory,
        collector=collector,
        llm_collector=llm_collector,

        user_message=user_message,
    )
    ai_message = collector.ai_post_response
    dataframe : pd.DataFrame | None = collector.dataframe_response
    memory.add_ai_message(ai_message, dataframe)
    

In [10]:
# Iniciando collector
collector = AppDataCollector()
llm_collector = LLMResponseCollector()
# memory.clear_memory()

chat("Now I want the static pressure", memory, collector, llm_collector)

generate-request
{'text': '\nintention:  The human is seeking the static pressure.'} 


request-type
{'text': "\ntype:  complex\nanalysis: I see this is a general question about a measurement system property, but I don't have access to the measurement system database, so this type is complex."} 


complex_request_process_modification
summary-conversation
{'text': '\nnew_summary:  The conversation is about a human asking the assistant for temperature and pressure data, specifically the average temperature for EMED-3138.12-050 in August 2023, and now requesting the static pressure.'} 


technical-terms
{'text': '\nterms: static pressure, temperature, system, EMED-3138.12-050'} 


enhanced-request-conversation
{'text': '\nresponse:  The human is requesting a summary of the previous conversation, specifically the request for static pressure information, which is relevant to the measurement system mentioned as EMED-3138.12-050.'} 


has-multi-definition
{'text': '\nclass: unclear\nanalysis:

C:\Users\lauth\OneDrive\Desktop\sql_assistant_v3\src\utils\sql_utils.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


summary-response-sql
{'text': 'response: The static pressure of the measurement system EMED-3138.12-050 is 367.521.'} 


post-process-translation
{'text': 'detected_language: English\nresponse: The static pressure of the measurement system EMED-3138.12-050 is 367.521.'} 




In [38]:
print(len(llm_collector.llm_responses))
print(llm_collector.llm_responses[12].prompt) #! what is the human currently requesting [4]

14
I need your help. Follow carefully the next steps:

First, look up the next user request:
user_request: '''"The human is requesting the static pressure of the measurement system EMED-3138.12-050 previously discussed."'''

Second, the next is a pandas dataframe that answers the previous request. Pay attention to this information:

|    |   Static Pressure | Measurement System Tag   |
|---:|------------------:|:-------------------------|
|  0 |           367.521 | EMED-3138.12-050         |
|  1 |           387.26  | EMED-3138.12-050         |
|  2 |           372.419 | EMED-3138.12-050         |
|  3 |           367.056 | EMED-3138.12-050         |
|  4 |           350.407 | EMED-3138.12-050         |
|  5 |           323.005 | EMED-3138.12-050         |
|  6 |           338.355 | EMED-3138.12-050         |
|  7 |           338.145 | EMED-3138.12-050         |
|  8 |           363.991 | EMED-3138.12-050         |
|  9 |           377.704 | EMED-3138.12-050         |

Third, generatio

In [ ]:
import json
print(collector.semantic_info.keys())
print(json.dumps(collector.terms_dictionary, indent=2))

In [13]:
print(memory.get_chat_history_lines(memory.chat_memory))


Human line: Give me the average temperature for EMED-3138.12-050 in august 2023
Assistant line: The average temperature for EMED-3138.12-050 in August 2023 is 23.502.
Assistant DataFrame: 
|    |   Average Temperature |
|---:|----------------------:|
|  0 |                23.502 |
Human line: Now I want the static pressure
Assistant line: The static pressure of the measurement system EMED-3138.12-050 is 367.521.
Assistant DataFrame: 
|    |   Static Pressure | Measurement System Tag   |
|---:|------------------:|:-------------------------|
|  0 |           367.521 | EMED-3138.12-050         |
|  1 |           387.26  | EMED-3138.12-050         |
|  2 |           372.419 | EMED-3138.12-050         |
|  3 |           367.056 | EMED-3138.12-050         |
|  4 |           350.407 | EMED-3138.12-050         |
|  5 |           323.005 | EMED-3138.12-050         |
|  6 |           338.355 | EMED-3138.12-050         |
|  7 |           338.145 | EMED-3138.12-050         |
|  8 |           363.

In [29]:
memory.add_ai_message("hi", None)